<a href="https://colab.research.google.com/github/hscrown/madatpublicdata/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C%EC%9D%84_%EC%9C%84%ED%95%9C_DB_%EC%84%A4%EA%B3%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DB 스키마

CREATE TABLE placeTable (
    placeID INT PRIMARY KEY,
    placeName VARCHAR(255),
    keyword VARCHAR(255) DEFAULT '강남구 도서관',
    placeType INT,  -- 0: 도서관, 1: 박물관, 2: 공원
    guName VARCHAR(50) DEFAULT '강남구'
);

CREATE TABLE userTable (
    userID INT PRIMARY KEY,
    placeID INT,
    reviewText TEXT,
    rating INT CHECK (rating >= 1 AND rating <= 5),
    FOREIGN KEY (placeID) REFERENCES placeTable(placeID)
);

In [28]:
import pandas as pd

In [29]:
# json 파일 불러오기
df = pd.read_json('reviews_240429.json', orient='records', lines=True, encoding='utf-8')

In [30]:
df.tail()

,User Name,Rating,URL,Review Text,Place Name,User Id,Place Id
4470,cxncnrienf288r,1,https://place.map.kakao.com/729961917,컴플레인은 중랑구청 도서관 담당하는 부서에 하면 직빵입니다. 여기다 하면 제식구 감...,중랑상봉도서관,482,159
4471,soo,5,https://place.map.kakao.com/729961917,,중랑상봉도서관,717,159
4472,ندى,5,https://place.map.kakao.com/729961917,새 건물 새 책 좋아요!,중랑상봉도서관,785,159
4473,🐰,1,https://place.map.kakao.com/729961917,도서관 문 연지 얼마 안됐을때 도서관 오후 8시에 닫는 걸로 공지되어있던데 7시 4...,중랑상봉도서관,2978,159
4474,Leeh,5,https://place.map.kakao.com/729961917,,중랑상봉도서관,272,159


### **1. userTable : 사용자 평점 정보가 담긴 테이블**

- **`userID`**: 사용자 식별 ID (기본 키)
- **`placeID`**: 사용자가 리뷰한 장소의 ID (외래 키, **`placeTable`**의 **`placeID`** 참조)
- **`reviewText`**: 사용자의 리뷰 내용
- **`rating`**: 장소에 대한 사용자의 평점 (1-5점)

In [31]:
userTable = df[['User Id', 'Place Id', 'Review Text', 'Rating']]

# 컬럼명 변경
userTable.columns = ['userID', 'placeID', 'reviewText', 'rating']

In [32]:
userTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4475 entries, 0 to 4474
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   userID      4475 non-null   int64 
 1   placeID     4475 non-null   int64 
 2   reviewText  4475 non-null   object
 3   rating      4475 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 140.0+ KB


In [33]:
# review text가 값이 '' 인것에 0값넣기
userTable.loc[userTable['reviewText'] == '', 'reviewText'] = 0.0

In [34]:
#  review text가 값이 0 이면 행삭제
userTable = userTable[userTable['reviewText'] != 0.0]

In [35]:
# 중복 삭제
userTable.drop_duplicates(inplace=True)

<ipython-input-35-5c0648149277>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  userTable.drop_duplicates(inplace=True)


In [36]:
userTable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3023 entries, 0 to 4473
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   userID      3023 non-null   int64 
 1   placeID     3023 non-null   int64 
 2   reviewText  3023 non-null   object
 3   rating      3023 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 118.1+ KB


In [37]:
userTable.reset_index(drop=True, inplace=True)

In [38]:
userTable.to_csv('userTable.csv', index=False, encoding='utf-8')
userTable.to_json('userTable.json', orient='records', lines=True)

### **2. placeTable : 장소 id와 이름, 키워드, 유형이 담긴 테이블**

- **`placeID`**: 장소 식별 ID (기본 키)
- **`placeName`**: 장소의 이름
- **`keyword`**: 장소와 관련된 키워드. 디폴트는 "강남구 도서관"
- **`placeType`** : 장소의 유형. 도서관은 0, 박물관은 1, 공원은 2로 표현.
- **`guName`** :  장소의 구 이름. 디폴트는 강남구.

#### df에서 place id, place name 추출

In [39]:
placeTable = df[['Place Id', 'Place Name']]

In [40]:
placeTable.drop_duplicates(inplace=True)

<ipython-input-40-6a4598915d25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  placeTable.drop_duplicates(inplace=True)


In [41]:
placeTable.reset_index(drop=True, inplace=True)

In [42]:
placeTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Place Id    178 non-null    int64 
 1   Place Name  178 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.9+ KB


#### keyword 가져오기

In [43]:
# json파일 읽기
df_keyword = pd.read_json('keywords_reviews.json', orient='lines', encoding='utf-8')

In [44]:
df_keyword.head()

,Place Id,Review,Keyword
0,0,설명 잘해주심 알차게 잘 구성해놨어요 서울시공공서비스예약 사이트에서 해설 신청하...,"[전시, 예약, 박물관, 특별, 사이트]"
1,1,공간도 넓고 정책 관련 도서도 많아요 자연과학열람터는 주말에는 열리지 않아요 사...,"[산책, 공부, 시험, 예약, 분위기]"
2,2,좋아요 책도 많고 깨끗합니다 웬만한 스카보다 좋음 넓고 좋다 주차장이 최악임 들...,"[독서실, 카페, 상태, 좋아요, 불만]"
3,3,장서량이 많지는 않지만 책보기 어려운 현대인에게 한줄기 빛 공공도서관은 사랑입니다 ...,"[사랑, 공부, 집중, 독서실, 최고]"
4,4,아담한 도서관 있을껀 다 있어요,"[도서관, 도서관, 아담한]"


In [45]:
# ['Keyword'] 칼럼의 내용을 리스트가 아닌 공백(' ')으로 구분된 문자열로 바꾸기
df_keyword['Keyword'] = df_keyword['Keyword'].apply(lambda x: ' '.join(x))

In [46]:
df_keyword.tail()

,Place Id,Review,Keyword
169,173,산 좋아 한적하니 좋아요 강아지산책시키기 좋아요,강아지 산책 강아지 좋아요 좋아요
170,174,두번째 방문 아이들이 직접 혈자리도 눌러보고 퀴즈도 풀어보는등 다양한 체험이 가능해...,무료 전시 구경 할인 체험
171,175,나중에 또 방문해서 해설사님 설명들으며 관람해보려구요 그냥 아담했음 옛집 속으...,아름답다 박물관 문화재 기둥 해설
172,176,현재기준 리모델링 중이라서 사용 불가 층에 주식하면서 전화 엄청 하...,분위기 동네 깔끔하다 독서 운동
173,177,내최애 작가 환기님과 향안여사님을 뵙다 도판으론 알수없다는 환기님의 작품 실물을...,전시 힐링 체험 추천 최고


In [47]:
df_keyword.loc[df_keyword['Place Id']==4,'Keyword'] = '아담한 도서관'

In [48]:
# df_keyword의 Keyword칼럼을 placeid가 같은 걸 찾아서 placeTable에 조인
placeTable = pd.merge(placeTable, df_keyword, on='Place Id', how='left')

In [49]:
placeTable.head()

,Place Id,Place Name,Review,Keyword
0,34,길동생태공원,현장 공원 주차 관리인들의 마인드가 갑인 미친 조직 발신 안녕하십니까 ...,산책 걷기 예약 공원 자연
1,76,서울대공원,가족끼리 놀러오기 좋은곳 주차공간도 널널하고 볼거리도 많고 뭣보다 동물원은 입장...,산책 여름 만족 연인 평화
2,81,서울숲,튤립 끝났어요 다들 활짝피거나 죽어서 절반이상이 누워있고 아예 잎 다 떨어진튤...,산책 이쁘다 무료 걷기 아름답다
3,142,월드컵공원,굿 좋습니다 산책하고 자전거 타기 너무 좋네요 시원하고 탁 트여서 좋아요 밤에는 호...,산책 여름 최고 공원 분위기
4,147,응봉공원,어르신들이 운동하시러 많이 오셨던 걍 공원이 아니에요 언덕이랑 계단 오져요 운동...,추천 공원 언덕 운동 어르신


In [50]:
placeTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Place Id    178 non-null    int64 
 1   Place Name  178 non-null    object
 2   Review      174 non-null    object
 3   Keyword     174 non-null    object
dtypes: int64(1), object(3)
memory usage: 5.7+ KB


In [51]:
placeTable.sample(10)

,Place Id,Place Name,Review,Keyword
159,93,서울특별시교육청남산도서관,멋진 곳 구석진 곳에서 할배들 샤우팅 노래배틀 열리면 하루는 그냥 공쳤다고 보시면 ...,산책 여름 친구 공부 힐링
10,162,천호근린공원,가을에 가도 이쁜곳 봄에 가면 정말 예쁜 곳,가을 가을 이쁜 예쁜
31,75,서울교육박물관,이번 특별전 무지 재밌어요 아이들 뿐만 아니라 어른들도 흥미롭게 볼만한 전시였습니...,무료 전시 특별 어른 시도
39,174,허준박물관,두번째 방문 아이들이 직접 혈자리도 눌러보고 퀴즈도 풀어보는등 다양한 체험이 가능해...,무료 전시 구경 할인 체험
99,140,우장산숲속도서관,나쁘진 않은데 사람이 도서관 크기에 비해 너무 많음 책 정리가 이렇게도 안된 ...,크기 정리 비슷비슷 번호 도서
121,95,서울특별시교육청도봉도서관,별로에요 책도 맨날 자리에 없고 층 열람실은 관리도 안되고 직원분들도 ...,카드 연장 처음 열람 자리
155,58,문래도서관,다른실은 다 시설 빵빵한데 일반열람실만 최악 의자 좀 리뉴얼하면 안되는지 골...,공부 집중 파서 실만 존경
148,37,꿈꾸는도서관,층에 애기들 많이서 좀 시끄럽긴 한데 그래도 책 많고 좋아요 좋다 ...,좋아요 사람 전화 애기 싸가지
92,89,서울특별시교육청강동도서관,노후되었지만 그래도 있을건 다있음 독서실처럼 돼있는데 냄새 역대급 홀애비냄새 ...,공부 휴게실 독서실 에어컨 예약
135,25,국립중앙도서관,날 좋을때 책 읽고 벤치에 누워서 광합성 하기 좋음 최고 수차례 평일 이용시간 연장...,만족 친절 공부 예약 최고


In [52]:
placeTable.loc[placeTable['Place Id'] == 26, 'Keyword'] = '용산구 박물관'

In [53]:
placeTable[placeTable['Place Id'] == 26]

,Place Id,Place Name,Review,Keyword
158,26,국립중앙박물관도서관,,용산구 박물관


주소를 키워드화 하기위해 csv 파일업로드

In [54]:
df_address = pd.read_csv('places.csv', encoding='utf-8')

In [55]:
df_address.sample(100)

,place_name,gu_name,type,place_id
1447,새마을문고망우3동분회작은도서관,중랑구,lib,1447
674,상도3동작은도서관,동작구,lib,674
1270,상림책마루작은도서관,은평구,lib,1270
1300,지웰작은도서관,은평구,lib,1300
446,한국산업기술시험원도서관,구로구,lib,446
...,...,...,...,...
144,미아문화정보도서관,강북구,lib,144
1008,더믹스만드는작은도서관,송파구,lib,1008
1122,책읽는쉼터,양천구,lib,1122
1115,연세 작은도서관,양천구,lib,1115


In [56]:
# keyword칼럼 lib은0, muse은 1,park은 2로 바꾸기
df_address['type2'] = df_address['type'].map({'lib':"도서관", 'muse':"박물관", 'par':"공원"})

In [57]:
# gu_name 컬럼과 type 컬럼의 문자열을 합친 keyword칼럼 만들기
df_address['keyword'] = df_address['gu_name'] + ' ' + df_address['type2']

In [58]:
# type칼럼 lib은0, muse은 1,park은 2로 바꾸기
df_address['type'] = df_address['type'].map({'lib':0, 'muse':1, 'par':2})

In [59]:
df_address.tail()
# type2 컬럼 삭제
df_address.drop(['type2'], axis=1, inplace=True)
df_address.tail()


,place_name,gu_name,type,place_id,keyword
1493,낙산근린공원,종로구,2,1493,종로구 공원
1494,양재시민의숲,서초구,2,1494,서초구 공원
1495,서울창포원,도봉구,2,1495,도봉구 공원
1496,북서울꿈의숲,강북구,2,1496,강북구 공원
1497,서서울호수공원,양천구,2,1497,양천구 공원


In [60]:
placeTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Place Id    178 non-null    int64 
 1   Place Name  178 non-null    object
 2   Review      174 non-null    object
 3   Keyword     174 non-null    object
dtypes: int64(1), object(3)
memory usage: 5.7+ KB


In [61]:
# 컬럼명 변경
df_address.columns = ['placeName','guName','placeType','placeID','Keyword']
# place id 삭제
df_address.drop(['placeID'], axis=1, inplace=True)

In [62]:
df_address.head()

,placeName,guName,placeType,Keyword
0,BIBLIOTECA,강남구,0,강남구 도서관
1,LH강남3단지작은도서관,강남구,0,강남구 도서관
2,休+밤토리SH작은도서관,강남구,0,강남구 도서관
3,강남구립못골도서관,강남구,0,강남구 도서관
4,강남데시앙파크작은도서관,강남구,0,강남구 도서관


In [63]:
placeTable
# 컬럼명 변경
placeTable.columns = ['placeID', 'placeName', 'Review', 'Keyword']

In [64]:
placeTable2 = placeTable.copy()

In [65]:
placeTable.head()

,placeID,placeName,Review,Keyword
0,34,길동생태공원,현장 공원 주차 관리인들의 마인드가 갑인 미친 조직 발신 안녕하십니까 ...,산책 걷기 예약 공원 자연
1,76,서울대공원,가족끼리 놀러오기 좋은곳 주차공간도 널널하고 볼거리도 많고 뭣보다 동물원은 입장...,산책 여름 만족 연인 평화
2,81,서울숲,튤립 끝났어요 다들 활짝피거나 죽어서 절반이상이 누워있고 아예 잎 다 떨어진튤...,산책 이쁘다 무료 걷기 아름답다
3,142,월드컵공원,굿 좋습니다 산책하고 자전거 타기 너무 좋네요 시원하고 탁 트여서 좋아요 밤에는 호...,산책 여름 최고 공원 분위기
4,147,응봉공원,어르신들이 운동하시러 많이 오셨던 걍 공원이 아니에요 언덕이랑 계단 오져요 운동...,추천 공원 언덕 운동 어르신


In [66]:
import numpy as np

In [67]:
# df_address와 placeTable을 placeName을 기준으로 합치기
merged_df = pd.merge(df_address, placeTable, on='placeName', how='left', suffixes=('_address', '_placeTable'))

# Keyword 컬럼의 값이 NaN일 때 df_address의 값을 사용
merged_df['Keyword'] = np.where(merged_df['Keyword_placeTable'].isna(), merged_df['Keyword_address'], merged_df['Keyword_placeTable'])

# 불필요한 컬럼 삭제
merged_df.drop(columns=['Keyword_address', 'Keyword_placeTable'], inplace=True)

# 결과 확인
print(merged_df.head())

      placeName guName  placeType  placeID Review  Keyword
0    BIBLIOTECA    강남구          0      NaN    NaN  강남구 도서관
1  LH강남3단지작은도서관    강남구          0      NaN    NaN  강남구 도서관
2  休+밤토리SH작은도서관    강남구          0      NaN    NaN  강남구 도서관
3     강남구립못골도서관    강남구          0      NaN    NaN  강남구 도서관
4  강남데시앙파크작은도서관    강남구          0      NaN    NaN  강남구 도서관


In [68]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   placeName  1498 non-null   object 
 1   guName     1498 non-null   object 
 2   placeType  1498 non-null   int64  
 3   placeID    107 non-null    float64
 4   Review     104 non-null    object 
 5   Keyword    1498 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 70.3+ KB


In [69]:
merged_df.drop(columns=['Review'], inplace=True)


In [70]:
merged_df.tail()

,placeName,guName,placeType,placeID,Keyword
1493,낙산근린공원,종로구,2,NaN,종로구 공원
1494,양재시민의숲,서초구,2,NaN,서초구 공원
1495,서울창포원,도봉구,2,NaN,도봉구 공원
1496,북서울꿈의숲,강북구,2,64.0,산책 평화 가을 힐링 강아지
1497,서서울호수공원,양천구,2,73.0,산책 걷기 평화 친구 가을


In [71]:
# place type의 값을 정수로 변경
merged_df['placeType'] = df_address['placeType'].astype(int)

In [72]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   placeName  1498 non-null   object 
 1   guName     1498 non-null   object 
 2   placeType  1498 non-null   int64  
 3   placeID    107 non-null    float64
 4   Keyword    1498 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 58.6+ KB


In [ ]:
# place id

In [73]:
df = merged_df.copy()

In [74]:
df.tail()

,placeName,guName,placeType,placeID,Keyword
1493,낙산근린공원,종로구,2,NaN,종로구 공원
1494,양재시민의숲,서초구,2,NaN,서초구 공원
1495,서울창포원,도봉구,2,NaN,도봉구 공원
1496,북서울꿈의숲,강북구,2,64.0,산책 평화 가을 힐링 강아지
1497,서서울호수공원,양천구,2,73.0,산책 걷기 평화 친구 가을


In [75]:
# 기존의 최대 placeID 값 찾기
max_place_id = df['placeID'].max()

# NaN인 placeID에 고유번호 부여
if np.isnan(max_place_id):
    max_place_id = 1  # 만약 모든 placeID가 NaN이면 1부터 시작
else:
    max_place_id += 1

# NaN 값을 새로운 ID로 대체
df['placeID'] = df['placeID'].fillna(value=pd.Series(range(int(max_place_id), int(max_place_id) + len(df))))
print(df)

         placeName guName  placeType  placeID          Keyword
0       BIBLIOTECA    강남구          0    177.0          강남구 도서관
1     LH강남3단지작은도서관    강남구          0    178.0          강남구 도서관
2     休+밤토리SH작은도서관    강남구          0    179.0          강남구 도서관
3        강남구립못골도서관    강남구          0    180.0          강남구 도서관
4     강남데시앙파크작은도서관    강남구          0    181.0          강남구 도서관
...            ...    ...        ...      ...              ...
1493        낙산근린공원    종로구          2   1670.0           종로구 공원
1494        양재시민의숲    서초구          2   1671.0           서초구 공원
1495         서울창포원    도봉구          2   1672.0           도봉구 공원
1496        북서울꿈의숲    강북구          2     64.0  산책 평화 가을 힐링 강아지
1497       서서울호수공원    양천구          2     73.0   산책 걷기 평화 친구 가을

[1498 rows x 5 columns]


In [76]:
df

,placeName,guName,placeType,placeID,Keyword
0,BIBLIOTECA,강남구,0,177.0,강남구 도서관
1,LH강남3단지작은도서관,강남구,0,178.0,강남구 도서관
2,休+밤토리SH작은도서관,강남구,0,179.0,강남구 도서관
3,강남구립못골도서관,강남구,0,180.0,강남구 도서관
4,강남데시앙파크작은도서관,강남구,0,181.0,강남구 도서관
...,...,...,...,...,...
1493,낙산근린공원,종로구,2,1670.0,종로구 공원
1494,양재시민의숲,서초구,2,1671.0,서초구 공원
1495,서울창포원,도봉구,2,1672.0,도봉구 공원
1496,북서울꿈의숲,강북구,2,64.0,산책 평화 가을 힐링 강아지


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   placeName  1498 non-null   object 
 1   guName     1498 non-null   object 
 2   placeType  1498 non-null   int64  
 3   placeID    1498 non-null   float64
 4   Keyword    1498 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 58.6+ KB


In [78]:
# csv파일로 저장
df.to_csv('placeTable.csv', index=False,encoding='utf-8')
# json파일로 저장
df.to_json('placeTable.json', orient='records')